In [2]:
import pandas as pd

In [3]:
dtype_dict={}
for x in ['user_id', 'retweeted_id', 'retweeted_user_id', 'id']:
    dtype_dict[x]=str


In [4]:
# Jean-Philippe
#df=pd.read_csv('../data/tweets_of_oncologist_and_advocacy.csv',dtype=dtype_dict)

# Aymeric
df = pd.read_csv("/home/aymeric/python-scripts/espadon/my-espadon-notebook/tweets_and_retweets_of_advocacy_oncologist.csv", 
                 sep=",", dtype = dtype_dict)

/tmp/ipykernel_253682/2831012075.py:5: DtypeWarning: Columns (4,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/aymeric/python-scripts/espadon/my-espadon-notebook/tweets_and_retweets_of_advocacy_oncologist.csv",


## Extraction des tweets des oncologues et des advocacies

Je n'ai pas compris à quoi servent les lignes de codes ci-dessous? Tu vas les réutiliser plus tard?

In [ ]:
dico_user_id_2_status=dict(zip(df.user_id,df.User_status))

In [ ]:
df['retweeted_user_status']=df['retweeted_user_id'].map(dico_user_id_2_status.get)

In [ ]:
df['retweeted_user_status'].value_counts()

In [ ]:
df['retweeted_id'].isna().sum()/len(df)

In [ ]:
df['User_status'].value_counts()

In [ ]:
df_adv=df[df['User_status']=='Advocacy']#.sample(50000)
df_onc=df[df['User_status']=='Oncologist']#.sample(50000)

In [ ]:
#df_adv.retweeted_user_status.value_counts()

In [ ]:
df[['User_status','retweeted_user_status']]

In [ ]:
print(len(df[(df['User_status']=='Advocacy')]))
df_adv=df[(df['User_status']=='Advocacy')].sample(100000,random_state=23)# | (df['retweeted_user_status']=='Advocacy')].sample(50000)


In [ ]:
df_adv=df_adv[df_adv.retweeted_user_id.isna()].sample(50000,random_state=23)

In [ ]:
df_adv.User_status.value_counts()

In [ ]:
onc_retweeted_ids=df[(df['retweeted_user_status']=='Advocacy') & (df['User_status']=='Oncologist')]['retweeted_id'].values


In [ ]:
len(df_onc),len(onc_retweeted_ids)

In [ ]:
dico_id={}
for x in onc_retweeted_ids:
    dico_id[x]=True

In [ ]:
df_onc.id

In [ ]:
df_adv[df_adv['id'].isin(onc_retweeted_ids)]

In [ ]:
df_adv['star']=df_adv.id.map(dico_id.get)

In [ ]:
df_adv.star.value_counts()

In [ ]:
len(df_adv)

In [ ]:
df_adv=df_adv.sample(10000,random_state=23)

In [ ]:
df_adv_s=df_adv.drop_duplicates(subset=['text'])
len(df_adv_s)

## Définition de l'échantillon

Ici, je comprends que tu définis l'échantillon à partir du dataframe originale (nommé df) et non à partir des sous-corpus "df_adv" ou "df_onc"

In [109]:
dfl=df.sample(4000,random_state=23)

In [110]:
#!pip install cleantext

In [111]:
import cleantext #j'ai une erreur avec cleantext : module 'cleantext' has no attribute 'replace_urls'
import re
import nltk
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords 




In [112]:
# Fonction de nettoyage utilisée en attendant de régler le problème de cleantext
def clean_text(x):
    x = str(x)
    x = x.lower().strip()
    x = re.sub(r'\brt\b','', x) #Enlève les RT
    #x = re.sub(r'#[A-Za-z0-9]*', ' ', x) #Enlève les hashtag plus les mots
    x = re.sub(r'#', ' ', x) #Enlève les hashtag sans les mots
    x = re.sub(r'https*://.*', ' ', x)
    x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
    tokens = word_tokenize(x)
    x = ' '.join([w for w in tokens if not w.lower() in stop_words])
    x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
    x = re.sub(r'\d+', ' ', x)
    x = re.sub(r'\n+', ' ', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x

In [113]:
def tok(text):
    tt = TweetTokenizer()
    return [x for x in tt.tokenize(re.sub("@[A-Za-z0-9_]+","", cleantext.replace_urls(text, replace_with=""))) if len(x)>2 and len(x)<30]


def tok2(text):
    tt = TweetTokenizer()
    cleaned_text = clean_text(text)
    return [x for x in tt.tokenize(cleaned_text) if len(x)>2 and len(x)<30]

#REMOVE ONLY THE '#'NOT THE WORD AFTER
def remove_hashtag_sign(text):
    text = re.sub(r'#', '', text)
    return text



Je ne vois pas où tu fais appel à la fonction remove_hashtag_sign?

Par ailleurs, est-ce utile d'enlever les "RT" ?

In [114]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aymeric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aymeric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [115]:

#dfl['text_clean'] = dfl['text'].apply(lambda x:remove_hashtag_sign(x))
#Remove extra white spaces, punctuation and apply lower casing
#dfl['text_clean'] = dfl['text_clean'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


dfl['text_clean']=dfl['text'].apply(clean_text)
dfl.head()# il y a des caractères japonais



,user_id,user_screen_name,User_status,retweeted_id,retweeted_user,text,user_description,sum_nb_biom,id,timestamp_utc,local_time,retweet_count,retweeted_user_id,retweeted_timestamp_utc,retweeted_user_status,text_clean
1357001,31000651,teamoncology,Oncologist,NaN,NaN,病院で、複数の医療機器・装置と共に移動し操作・管理を行う≪臨床工学技士≫のお話をコラムに載せ...,"University of Hawai’i Cancer Center, Director....",77.0,237409610665955328,1345437836,2012-08-20T04:43:56,1,NaN,NaN,NaN,病院で、複数の医療機器・装置と共に移動し操作・管理を行う≪臨床工学技士≫のお話をコラムに載せ...
114580,904861723,abcdiagnosis,Advocacy,NaN,NaN,Exactly & this is what we want this to change ...,Founder of After Breast Cancer Diagnosis & liv...,60.0,730040390900981760,1462890167,2016-05-10T14:22:47,1,NaN,NaN,NaN,exactly want change help others datasaveslives...
1360829,31000651,teamoncology,Oncologist,NaN,NaN,♬ 'Hyakumankaino Aishiteru' - Oranje. ♪ この曲気に入...,"University of Hawai’i Cancer Center, Director....",77.0,136636552712437761,1321411667,2011-11-16T02:47:47,0,NaN,NaN,NaN,♬ 'hyakumankaino aishiteru ' oranje ♪ この曲気に入ってます
881406,14465456,MightyCasey,Advocacy,NaN,NaN,@ePatientDave @findlungcancer @MyelomaTeacher ...,Snarky #HealthData journo/wonk/activist. This ...,21.0,1039868733660049409,1536759000,2018-09-12T13:30:00,0,NaN,NaN,NaN,tagging peeps might knowledge
1787238,14062554,MDAndersonNews,Health center,NaN,NaN,@realsarahpepper Thank you for your support!,"MD Anderson is a global leader in cancer care,...",53.0,1188811231324442624,1572269659,2019-10-28T13:34:19,2,NaN,NaN,NaN,thank support


In [116]:
dfl['len']=dfl.text_clean.apply(len)

In [117]:
dfl=dfl[dfl.len>=9]
len(dfl)

3818

Pourquoi ci-dessous tu prends le texte original et pas le text nettoyé?

In [118]:
# df_adv_s=dfl.dropna(subset='text_clean')#df_adv_s.dropna(subset='text')
# je trouve le nom de la variable df_adv_s un peu confu, car df_adv est défini au début, je propose de remplacer par =
dfl_s=dfl.dropna(subset='text_clean')#df_adv_s.dropna(subset='text')

corpus= list(dfl_s.text.values)

len(corpus)

3818

In [119]:
#df_adv_s_export=df_adv_s[df_adv_s['text'].isin(docs)]

In [120]:
#df_adv_s_export.to_csv('sample_10k_text_espadon_advocacy.csv')
dfl_s.to_csv('sample_5k_text_espadon_.csv')

In [121]:
dico_doc_2_author=dict(zip(dfl_s.text,dfl_s.user_screen_name))

In [122]:
#topic_model = BERTopic(embedding_model=embedding_model)
#topic_model.fit_transform(corpus[:])
len(corpus)

3818

## Définition des paramètre de bertopic

In [123]:
from umap import UMAP
from hdbscan import HDBSCAN

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from sentence_transformers import SentenceTransformer



In [124]:
#! pip3 install emoji==0.6.0

In [125]:
# Step 1 - Extract embeddings
#embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
#embedding_model = SentenceTransformer("vinai/bertweet-base")
embedding_model = SentenceTransformer('digio/Twitter4SSE')


stopwords = stopwords.words('english') 

# Define sub-models
vectorizer = CountVectorizer(ngram_range=(1,1),stop_words='english')#,vocabulary=vocab_filtered)

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=12, min_samples=2, metric='euclidean', prediction_data=True,cluster_selection_method='eom')


No sentence-transformers model found with name /home/aymeric/.cache/torch/sentence_transformers/digio_Twitter4SSE. Creating a new one with MEAN pooling.


In [126]:
#docs=corpus[:10000]
docs=corpus

In [127]:
embeddings = embedding_model.encode(docs,show_progress_bar=True)


Batches:   0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
#stsb-distilbert-base

In [149]:
vectorizer = CountVectorizer(ngram_range=(1,2)) #vectorize est déjà défini une première fois ci-dessus, lequel garder
topic_model = BERTopic(
    #embedding_model=embedding_model, #
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer).fit(docs, embeddings)

In [150]:
legend=topic_model.get_topic_info()
legend

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1315,-1_cancer_patients_research_bcsm,"[cancer, patients, research, bcsm, asco, lcsm,...",[ great honor present study covid esmo monitor...
1,0,127,0_good_ditto_point_thorpe,"[good, ditto, point, thorpe, thinks, attitude,...",[good point previous future relationship mess ...
2,1,91,1_cells_via_melanoma_gene,"[cells, via, melanoma, gene, cd, immunotherapy...",[precisionmedicine precisiononcology « naturep...
3,2,82,2_covid_cases_enough_mask,"[covid, cases, enough, mask, coronavirus, hosp...",[enough honesty testing quarantine rules apply...
4,3,82,3_xx_thanks_aw_xxxx,"[xx, thanks, aw, xxxx, happy, thank, christmas...",[treated first wardrobe items chatham last wee...
...,...,...,...,...,...
87,86,12,86_nci_genomic_molecular testing_naclc,"[nci, genomic, molecular testing, naclc, et al...",[thks highlighting kettering study allohct mye...
88,87,12,87_eolchat_follow_family_follow up,"[eolchat, follow, family, follow up, astro, up...",[yup talking teachers dont know relate parents...
89,88,12,88_dx_living_yung overcoming_maintaining cancer,"[dx, living, yung overcoming, maintaining canc...",[t holidays stressful time yr add cancer dx tr...
90,89,12,89_data_systems_moc meaningful_marcsfz,"[data, systems, moc meaningful, marcsfz, meani...",[agree need data advocating blind usage resist...


Si je comprends bien, tu fais une première vectorisation en concervant les stopwords puis une seconde avec des textes nettoyés. L'intéret c'est quoi ? De calculer un embedding à partir de l'ensemble des éléments du textes et ensuite de les enlever de la représentation ?

In [131]:
 
vectorizer_model2 = CountVectorizer(ngram_range=(1,2),tokenizer=tok2,stop_words='english',min_df=10)
topic_model.update_topics(docs, vectorizer_model=vectorizer_model2)

In [132]:
# Generate nicer looking labels and set them in our model
topic_labels = topic_model.generate_topic_labels(nr_words=10,
                                                 topic_prefix=True,
                                                 word_length=20,
                                                 separator=", ")
topic_model.set_topic_labels(topic_labels)

In [133]:

legend=topic_model.get_topic_info()
legend

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,1315,-1_research_cancer_patients_trial,"-1, research, cancer, patients, trial, asco, b...","[research, cancer, patients, trial, asco, brea...",[ great honor present study covid esmo monitor...
1,0,127,0_point_good_twitter_started,"0, point, good, twitter, started, morning, loo...","[point, good, twitter, started, morning, look,...",[good point previous future relationship mess ...
2,1,91,1_melanoma_genetic_mutations_dna,"1, melanoma, genetic, mutations, dna, immunoth...","[melanoma, genetic, mutations, dna, immunother...",[precisionmedicine precisiononcology « naturep...
3,2,82,2_covid_low_said_medical,"2, covid, low, said, medical, using, risk, sym...","[covid, low, said, medical, using, risk, sympt...",[enough honesty testing quarantine rules apply...
4,3,82,3_thanks_happy_love_thank,"3, thanks, happy, love, thank, got, hope, yes,...","[thanks, happy, love, thank, got, hope, yes, m...",[treated first wardrobe items chatham last wee...
...,...,...,...,...,...,...
87,86,12,86_molecular_specific_clinical_nsclc,"86, molecular, specific, clinical, nsclc, test...","[molecular, specific, clinical, nsclc, testing...",[thks highlighting kettering study allohct mye...
88,87,12,87_family_think_believe_similar,"87, family, think, believe, similar, ways, hop...","[family, think, believe, similar, ways, hope, ...",[yup talking teachers dont know relate parents...
89,88,12,88_living_add_hope_life,"88, living, add, hope, life, endcancer, men, a...","[living, add, hope, life, endcancer, men, ask,...",[t holidays stressful time yr add cancer dx tr...
90,89,12,89_data_huge_similar_focus,"89, data, huge, similar, focus, treat, resista...","[data, huge, similar, focus, treat, resistance...",[agree need data advocating blind usage resist...


In [134]:
topicnb_2_name=dict(zip(legend['Topic'],legend['CustomName']))


In [135]:
fig=topic_model.visualize_barchart(topics=list(range(len(topic_model.get_topic_info())-1)),n_words=10)

fig.write_html(str(len(topic_model.get_topic_info()))+'.html')

In [136]:
nb_topic=len(legend)

In [137]:
# Generate nicer looking labels and set them in our model
topic_labels = topic_model.generate_topic_labels(nr_words=10,
                                                 topic_prefix=True,
                                                 word_length=20,
                                                 separator=", ")
topic_model.set_topic_labels(topic_labels)

In [138]:
topic_labels

['-1, research, cancer, patients, trial, asco, breast, treatment, lcsm, bcsm, life',
 '0, point, good, twitter, started, morning, look, check, like, say, hashtag',
 '1, melanoma, genetic, mutations, dna, immunotherapy, tumor, resistance, cell, myeloma, response',
 '2, covid, low, said, medical, using, risk, symptoms, care, testing, people',
 '3, thanks, happy, love, thank, got, hope, yes, morning, wow, old',
 '4, social, online, use, ask, questions, community, pts, disease, hashtag, health',
 '5, lymsm, second, link, hashtag, org, look, asco, twitter, lcsm, bcsm',
 '6, esmo, virtual, forward, join, looking, free, meeting, conference, latest, meet',
 '7, age, pts, medicine, trials, post, think, patient, point, perspective, term',
 '8, thanks, happy, recent, meet, great, sure, news, community, latest, wait',
 '9, family, opportunity, meet, making, education, home, hard, work, support, believe',
 '10, cell, doc, lymsm, lymphoma, treatments, myeloma, including, sabcs, diagnosed, surgery',


In [139]:
info_doc=topic_model.get_document_info(docs)
dico_doc_2_topic=dict(zip(info_doc.Document,info_doc.Topic))

In [140]:
#dico_doc_2_topic

In [141]:
#info_doc3=info_doc[info_doc['Topic']==3]
#info_doc3[info_doc3.Document.str.contains('@katie23085')].Document.values

In [144]:
import textwrap# Manually selected some interesting topics to prevent information overload
topics_of_interest = list(range(nb_topic))

def add_br(example,N=100):
    return '<br>'.join(l for line in example.splitlines() 
                  for l in textwrap.wrap(line, width=N))
    
    

adjusted_docs = ["<b>"+"Topic:"+str(dico_doc_2_topic[doc])+" - " + topicnb_2_name[dico_doc_2_topic[doc]] +"</b><br>"# add_br(doc[:]) +"<br><b>by " + 
                 #dico_doc_2_author[doc]+"</b>" 
                 for doc in docs]

# Visualize documents
fig=topic_model.visualize_documents(
    adjusted_docs, 
    embeddings=embeddings, 
    hide_annotations=True, 
    topics=topics_of_interest,
    custom_labels=True,
    width = 2000,
    height = 1300
)
fig.write_html(str(nb_topic)+'s.html')

In [ ]:
df_adv_s['text_clean']

In [ ]:
dfl['topic_nb']=dfl['text'].map(dico_doc_2_topic.get)
dfl['topic_label']=dfl['topic_nb'].map(topicnb_2_name.get)

In [ ]:
documentsm1=list(dfl[dfl.topic_nb!=-1][['topic_label','user_screen_name','User_status','text']].transpose().to_dict().values())
#documents

In [ ]:
dfl['topic_nb']

In [ ]:
from nomic import atlas


In [ ]:
embedm1=embeddings[dfl.topic_nb!=-1]
#documentsm1=documents[dfl.topic_nb!=-1]

In [ ]:
response = atlas.map_embeddings(embeddings=embedm1,
                                data=documentsm1,
                                colorable_fields=['topic_label'],
                                name="older#sm",add_datums_if_exists = True, build_topic_model=False)
                                #description="An example of building a text map with a huggingface model.")

print(response)

In [ ]:
#build_topic_model

In [ ]:
response = atlas.map_text(#embeddings=embeddings,
                                data=documents,
                                indexed_field='text',
                                colorable_fields=['topic_label'],
                                multilingual=True,
                                name="newest#sm#text",add_datums_if_exists = True, build_topic_model=True)
                                #description="An example of building a text map with a huggingface model.")

print(response)

In [ ]:
response.